# Step2按照不同队列间进行比较

In [ ]:
library(ropls)
library(ggsci)
library(Cairo)
library(tidyverse)
library(malbacR)
library(pmartR)
library(ggplot2)
library(dplyr)
library(tibble)
library(tidyr)
library(paletteer)
library(extrafont)
library(ggpubr)
library(eulerr)
my36colors <-c(
  "#1f77b4","#d62728","#ff7f0e","#2ca02c","#9467bd","#8c564b",
  "#e377c2","#7f7f7f","#17becf","#aec7e8","#ffbb78",
  "#98df8a","#ff9896","#c5b0d5","#c49c94","#f7b6d2","#c7c7c7",
  "#dbdb8d","#9edae5","#7698b3","#d6616b","#a55194","#ce6dbd",
  "#756bb1","#8c6d31","#b5cf6b","#7b4173","#cedb9c","#6b6ecf",
  "#9c9ede","#bd9e39","#d9d9d9","#ad494a","#8ca252","#e7ba52"
) 

In [ ]:
group_standard <- 'VFA'
save_file_prex <- 'wilcox_change_VFA_group2'
group_cohort <- 'Validation Cohort'

In [ ]:
path_save <- paste(
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_',group_standard,
    '_group2/',sep = ''
)
dir.create(path_save,showWarnings = FALSE)
path_save

#### 发现队列中基线和验证队列基线计算到的差异代谢物

In [ ]:
file_path <- paste(
    '/mnt/data3/data_exp_0826/result/Development_Cohort_wilcox_change_',group_standard,
    '_group2/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = ''
)
data_smi_develop <- read.csv(
    file_path,
    row.names = 1) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    )
data_smi_develop %>% head()

In [ ]:
file_path <- paste(
    '/mnt/data3/data_exp_0826/result/Validation_Cohort_wilcox_change_',group_standard,
    '_group2/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = ''
)
data_smi_validation <- read.csv(
    file_path,
    row.names = 1) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    )
data_smi_validation %>% head()

In [ ]:
level_set <- data_smi_develop$level %>% unique()
level_set

#### sarcopenia

In [ ]:
cat_use <- 'Up'
data_smi_develop_sub <- data_smi_develop %>% 
    filter(cat == cat_use) %>% #,level == level_use
    select(c('metabolites','log2FC','pvalue','vip')) %>% 
    rename_with(~ paste0(., "_develop"), -metabolites)
data_smi_validation_sub <- data_smi_validation %>% 
    filter(cat == cat_use) %>% #,level == level_use
    select(c('metabolites','level','log2FC','pvalue','cat','vip')) %>% 
    arrange(level,desc(abs(log2FC))) %>% 
    rename_with(~ paste0(., "_validation"), -c('metabolites','level','cat'))
data_plot <- list(
  `Developmental Cohort` = data_smi_develop_sub$metabolites %>% unique(),
  `Validation Cohort` = data_smi_validation_sub$metabolites %>% unique()
)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
par(mar = c(5, 100, 4, 2)) 
p_venn <- plot(
  euler(data_plot,shape = "ellipse"),
  quantities = list(
    type = c("percent", "counts"),
    col = "black",
    font = 2,
    cex = 2
  ),
  labels = list(
    labels = names(data_plot) %>% str_remove(' Cohort'),
    col = "black",
    font = 3,
    cex = 3
  ),
  edges = list(col = my36colors, lwd = 5, lty = 1),
  main = list(label=c(
      paste("Highly Expressed in ",group_standard,'-Decrease[change]',sep = '')
  ),cex=3),
  legend = list(
    labels = names(data_plot),
    font = 1,
    cex = 2,
    side = "right",
    # x = 10,
    x = 150,    
    y = 3
  )
)
p_venn

In [ ]:
cat_use <- 'Up'
data_smi_develop_sub <- data_smi_develop %>% 
    filter(cat == cat_use) 
    select(c('metabolites','log2FC','pvalue','vip')) %>% 
    rename_with(~ paste0(., "_develop"), -metabolites)
data_smi_validation_sub <- data_smi_validation %>% 
    filter(cat == cat_use) %>% 
    select(c('metabolites','level','log2FC','pvalue','cat','vip')) %>% 
    arrange(level,desc(abs(log2FC))) %>% 
    rename_with(~ paste0(., "_validation"), -c('metabolites','level','cat'))
data_res_up <- data_smi_validation_sub %>% 
    left_join(data_smi_develop_sub,by = 'metabolites') %>% 
    filter(!is.na(log2FC_develop)) %>% 
    select(c('metabolites','level','cat','log2FC_validation','pvalue_validation','vip_validation','log2FC_develop','pvalue_develop','vip_develop'))
data_res_up

#### non-sarcopenia

In [ ]:
cat_use <- 'Down'
data_smi_develop_sub <- data_smi_develop %>% 
    filter(cat == cat_use) %>% #,level == level_use
    select(c('metabolites','log2FC','pvalue','vip')) %>% 
    rename_with(~ paste0(., "_develop"), -metabolites)
data_smi_validation_sub <- data_smi_validation %>% 
    filter(cat == cat_use) %>% #,level == level_use
    select(c('metabolites','level','log2FC','pvalue','cat','vip')) %>% 
    arrange(level,desc(abs(log2FC))) %>% 
    rename_with(~ paste0(., "_validation"), -c('metabolites','level','cat'))
data_plot <- list(
  `Developmental Cohort` = data_smi_develop_sub$metabolites %>% unique(),
  `Validation Cohort` = data_smi_validation_sub$metabolites %>% unique()
)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
par(mar = c(5, 100, 4, 2))  
p_venn <- plot(
  euler(data_plot,shape = "ellipse"),
  quantities = list(
    type = c("percent", "counts"),
    col = "black",
    font = 2,
    cex = 2
  ),
  # fills = list(fill = my3color),
  labels = list(
    labels = names(data_plot) %>% str_remove(' Cohort'),
    col = "black",
    font = 3,
    cex = 3
  ),
  edges = list(col = my36colors, lwd = 5, lty = 1),
  main = list(label=c(
   paste("Highly Expressed in ",group_standard,'-Stable[change]',sep = '')   
  ),cex=3),
  legend = list(
    labels = names(data_plot),
    font = 1,
    cex = 2,
    side = "right",
    # x = 10,
    x = 150,     # 调整x位置
    y = 3
  )
)
p_venn

In [ ]:
cat_use <- 'Down'
data_smi_develop_sub <- data_smi_develop %>% 
    filter(cat == cat_use) %>% #,level == level_use
    select(c('metabolites','log2FC','pvalue','vip')) %>% 
    rename_with(~ paste0(., "_develop"), -metabolites)
data_smi_validation_sub <- data_smi_validation %>% 
    filter(cat == cat_use) %>% 
    select(c('metabolites','level','log2FC','pvalue','cat','vip')) %>% 
    arrange(level,desc(abs(log2FC))) %>% 
    rename_with(~ paste0(., "_validation"), -c('metabolites','level','cat'))
data_res_down <- data_smi_validation_sub %>% 
    left_join(data_smi_develop_sub,by = 'metabolites') %>% 
    filter(!is.na(log2FC_develop)) %>% 
    select(c('metabolites','level','cat','log2FC_validation','pvalue_validation','vip_validation','log2FC_develop','pvalue_develop','vip_develop'))
data_res_down

In [ ]:
data_res_down$metabolites %>% unique()

In [ ]:
.on.public.web = FALSE
CalculateHyperScore_self <- function (mSetObj = NA) {
    mSetObj <- MetaboAnalystR:::.get.mSet(mSetObj)
    if (mSetObj$analSet$type == "msetssp") {
        ora.vec <- mSetObj$dataSet$cmpd
    }else {
        nm.map <- GetFinalNameMap(mSetObj)
        valid.inx <- !(is.na(nm.map$hmdb) | duplicated(nm.map$hmdb))
        ora.vec <- nm.map$hmdb[valid.inx]
    }
    q.size <- length(ora.vec)
    if (all(is.na(ora.vec)) || q.size == 0) {
        AddErrMsg("No valid HMDB compound names found!")
        return(0)
    }
    if (!.on.public.web & grepl("kegg", mSetObj$analSet$msetlibname)) {
        if (!exists("my.hyperscore.kegg")) {
            .load.scripts.on.demand("util_api.Rc")
        }
        mSetObj$api$oraVec <- ora.vec
        if (mSetObj$api$filter) {
            mSetObj$api$filterData <- mSetObj$dataSet$metabo.filter.kegg
            toSend <- list(mSet = mSetObj, libNm = mSetObj$api$libname, 
                filter = mSetObj$api$filter, oraVec = mSetObj$api$oraVec, 
                filterData = mSetObj$api$filterData, excludeNum = mSetObj$api$excludeNum)
        }else {
            toSend <- list(mSet = mSetObj, libNm = mSetObj$api$libname, 
                filter = mSetObj$api$filter, oraVec = mSetObj$api$oraVec, 
                excludeNum = mSetObj$api$excludeNum)
        }
        saveRDS(toSend, "tosend.rds")
        return(my.hyperscore.kegg())
    }
    current.mset <- current.msetlib$member
    if (mSetObj$dataSet$use.metabo.filter && !is.null(mSetObj$dataSet$metabo.filter.hmdb)) {
        current.mset <- lapply(current.mset, function(x) {
            x[x %in% mSetObj$dataSet$metabo.filter.hmdb]
        })
        mSetObj$dataSet$filtered.mset <- current.mset
    }
    set.size <- length(current.mset)
    if (set.size == 1) {
        AddErrMsg("Cannot perform enrichment analysis on a single metabolite set!")
        return(0)
    }
    my.univ <- unique(unlist(current.mset, use.names = FALSE))
    if (!is.null(mSetObj$dataSet$metabo.filter.hmdb)) {
        my.univ <- unique(mSetObj$dataSet$metabo.filter.hmdb)
    }
    uniq.count <- length(my.univ)
    ora.vec <- ora.vec[ora.vec %in% my.univ]
    q.size <- length(ora.vec)
    hits <- lapply(current.mset, function(x) {
        x[x %in% ora.vec]
    })
    hit.num <- unlist(lapply(hits, function(x) length(x)), use.names = FALSE)
    if (sum(hit.num > 0) == 0) {
        AddErrMsg("No match was found to the selected metabolite set library!")
        return(0)
    }
    set.num <- unlist(lapply(current.mset, length), use.names = FALSE)
    res.mat <- matrix(NA, nrow = set.size, ncol = 6)
    rownames(res.mat) <- names(current.mset)
    colnames(res.mat) <- c("total", "expected", "hits", "Raw p", 
        "Holm p", "FDR")
    for (i in 1:set.size) {
        res.mat[i, 1] <- set.num[i]
        res.mat[i, 2] <- q.size * (set.num[i]/uniq.count)
        res.mat[i, 3] <- hit.num[i]
        res.mat[i, 4] <- phyper(hit.num[i] - 1, set.num[i], uniq.count - 
            set.num[i], q.size, lower.tail = F)
    }
    res.mat[, 5] <- p.adjust(res.mat[, 4], "holm")
    res.mat[, 6] <- p.adjust(res.mat[, 4], "fdr")
    res.mat <- res.mat %>% as.data.frame() %>% filter(hits > 0) %>% arrange(desc(names(.)[4]))
    # res.mat <- res.mat[hit.num > 0, ]
    ord.inx <- order(res.mat[, 4])
    mSetObj$analSet$ora.mat <- signif(res.mat[ord.inx, ], 3)
    mSetObj$analSet$ora.hits <- hits
    MetaboAnalystR:::fast.write.csv(mSetObj$analSet$ora.mat, file = "msea_ora_result.csv")
    return(MetaboAnalystR:::.set.mSet(mSetObj))
}
func_metaboanalisys <- function (tmp.vec, pb_use = "smpdb_pathway", p_use = "Raw p") {
    library(MetaboAnalystR)
    mSet <- InitDataObjects("conc", "msetora", FALSE)
    mSet <- Setup.MapData(mSet, tmp.vec)
    mSet <- CrossReferencing(mSet, "name")
    mSet$name.map
    mSet <- CreateMappingResultTable(mSet)
    if (!grepl("kegg", pb_use, ignore.case = TRUE)) {
        mSet <- SetMetabolomeFilter(mSet, F)
        data_res <- SetCurrentMsetLib(mSet, pb_use, 0)
        data_res <- CalculateHyperScore_self(data_res)
        mSetObj <- data_res
        res_metabolites <- lapply(1:length(mSetObj$analSet$ora.hits),FUN = function(x){
            if(length(mSetObj$analSet$ora.hits[[x]])>0){
                metabolites_all <- mSetObj$analSet$ora.hits[[x]]
                pathwayname <- mSetObj$analSet$ora.hits[x] %>% names()
                tmp <- data.frame(
                    pathwayname = pathwayname,
                    metabolites_all = paste(metabolites_all,collapse = ';')
                )
                return(tmp)
            }
        }) %>% Filter(Negate(is.null), .) %>% 
            do.call(rbind,.)
        folds <- mSetObj$analSet$ora.mat[, 3]/mSetObj$analSet$ora.mat[,2]
        names(folds) <- MetaboAnalystR:::GetShortNames(rownames(mSetObj$analSet$ora.mat))
        pvals <- mSetObj$analSet$ora.mat[, 4]
        data_plot <- data.frame(pathway = folds %>% names() %>% 
            as.factor(), enrichment_ratio = folds %>% as.numeric(), 
            hits = mSetObj$analSet$ora.mat[, 3], pvalue = pvals) %>% 
            arrange(enrichment_ratio) %>% mutate(pathway = pathway %>% 
            factor(levels = pathway %>% unique()), pvalue = -log10(pvalue))
        data_plot %>% head()
        value_min <- data_plot[[ncol(data_plot)]] %>% min() %>% 
            {
                . * 100
            } %>% ceiling() %>% {
            ./100
        }
        value_max <- data_plot[[ncol(data_plot)]] %>% max() %>% 
            {
                . * 100
            } %>% floor() %>% {
            ./100
        }
        value_middle <- (value_min + (value_max - value_min)/2) %>% 
            round(., digits = 1)
        plot_res <- ggplot(data = data_plot, aes(x = enrichment_ratio, 
            y = pathway, fill = .data[[names(data_plot)[ncol(data_plot)]]])) + 
            geom_bar(stat = "identity") + geom_text(aes(x = enrichment_ratio + 
            max(enrichment_ratio) * 0.05, label = hits), size = 10, 
            hjust = 1) + geom_text(data = data_plot[data_plot$pvalue > 
            (-log10(0.05)), ], aes(x = enrichment_ratio + max(enrichment_ratio) * 
            0.05, label = hits), color = "red", size = 10, hjust = 1) + 
            scale_x_continuous(expand = c(0.01, 0)) + scale_fill_gradient2(low = "white", 
            mid = "#AAC0CF", high = "#2B5C8A", breaks = c(value_min, 
                value_middle, value_max)) + labs(fill = "-Log10(Pvalue)") + 
            theme_classic() + theme(plot.margin = margin(l = 10), 
            axis.text = element_text(size = 20), axis.text.y = element_text(size = 30, 
                angle = 0, hjust = 1, vjust = 0.5), axis.title = element_text(size = 24, 
                hjust = 0.5, vjust = 1), axis.title.y = element_blank(), 
            legend.position = "right", legend.title = element_text(size = 24, 
                margin = margin(b = 20)), legend.text = element_text(size = 20), 
            )
    }
    else {
        mSet <- InitDataObjects("conc", "msetora", FALSE)
        mSet <- Setup.MapData(mSet, tmp.vec)
        mSet <- CrossReferencing(mSet, "name")
        mSet <- CreateMappingResultTable(mSet)
        mSet <- SetMetabolomeFilter(mSet, F)
        nm.map <- GetFinalNameMap(mSet)
        valid.inx <- !(is.na(nm.map$hmdb) | duplicated(nm.map$hmdb))
        ora.vec <- nm.map$hmdb[valid.inx]
        q.size <- length(ora.vec)
        if (all(is.na(ora.vec)) || q.size == 0) {
            AddErrMsg("No valid HMDB compound names found!")
            return(0)
        }
        list_kegg_all <- readRDS("/mnt/data/jupyter_code/代谢组学/全谱代谢组分析/list_kegg_all.rds")
        current.mset <- list_kegg_all
        set.size <- length(current.mset)
        if (set.size == 1) {
            AddErrMsg("Cannot perform enrichment analysis on a single metabolite set!")
            return(0)
        }
        set.size <- length(current.mset)
        if (set.size == 1) {
            AddErrMsg("Cannot perform enrichment analysis on a single metabolite set!")
            return(0)
        }
        my.univ <- unique(unlist(current.mset, use.names = FALSE))
        if (!is.null(mSet$dataSet$metabo.filter.hmdb)) {
            my.univ <- unique(mSet$dataSet$metabo.filter.hmdb)
        }
        uniq.count <- length(my.univ)
        ora.vec <- ora.vec[ora.vec %in% my.univ]
        q.size <- length(ora.vec)
        hits <- lapply(current.mset, function(x) {
            x[x %in% ora.vec]
        })
        #比对的分子导出
        res_metabolites <- lapply(1:length(hits),FUN = function(x){
            if(length(hits[[x]])>0){
                metabolites_all <- hits[[x]]
                pathwayname <- hits[x] %>% names()
                tmp <- data.frame(
                    pathwayname = pathwayname,
                    metabolites_all = paste(metabolites_all,collapse = ';')
                )
                return(tmp)
            }
        }) %>% Filter(Negate(is.null), .) %>% 
            do.call(rbind,.)
        hit.num <- unlist(lapply(hits, function(x) length(x)), 
            use.names = FALSE)
        if (sum(hit.num > 0) == 0) {
            AddErrMsg("No match was found to the selected metabolite set library!")
            return(0)
        }
        set.num <- unlist(lapply(current.mset, length), use.names = FALSE)
        res.mat <- matrix(NA, nrow = set.size, ncol = 6)
        rownames(res.mat) <- names(current.mset)
        colnames(res.mat) <- c("total", "expected", "hits", "Raw p", 
            "Holm p", "FDR")
        for (i in 1:set.size) {
            res.mat[i, 1] <- set.num[i]
            res.mat[i, 2] <- q.size * (set.num[i]/uniq.count)
            res.mat[i, 3] <- hit.num[i]
            res.mat[i, 4] <- phyper(hit.num[i] - 1, set.num[i], 
                uniq.count - set.num[i], q.size, lower.tail = F)
        }
        res.mat[, 5] <- p.adjust(res.mat[, 4], "holm")
        res.mat[, 6] <- p.adjust(res.mat[, 4], "fdr")
        res.mat <- res.mat %>% as.data.frame() %>% filter(hits > 
            0) %>% arrange(desc(names(.)[4]))
        p_label <- ifelse(p_use == "Raw p", "Pvalue", "P.adj")
        data_plot <- signif(res.mat, 3) %>% as.data.frame() %>% 
            mutate(enrichment_ratio = hits/expected) %>% rownames_to_column("pathway") %>% 
            arrange(enrichment_ratio) %>% mutate(pathway = pathway %>% 
            str_remove_all(" - Homo sapiens \\(human\\)"), pathway = factor(pathway, 
            levels = pathway %>% unique()), `:=`(!!paste0("-log10(", 
            p_label, ")"), -log10(!!sym(p_use))))
        value_min <- data_plot[[ncol(data_plot)]] %>% min() %>% 
            {
                . * 100
            } %>% ceiling() %>% {
            ./100
        }
        value_max <- data_plot[[ncol(data_plot)]] %>% max() %>% 
            {
                . * 100
            } %>% floor() %>% {
            ./100
        }
        value_middle <- (value_min + (value_max - value_min)/2) %>% 
            round(., digits = 1)
        plot_res <- ggplot(data = data_plot, aes(x = enrichment_ratio, 
            y = pathway, fill = .data[[names(data_plot)[ncol(data_plot)]]])) + 
            geom_bar(stat = "identity") + geom_text(aes(x = enrichment_ratio + 
            max(enrichment_ratio) * 0.05, label = hits), size = 10, 
            hjust = 1) + geom_text(data = data_plot[data_plot[[names(data_plot)[ncol(data_plot)]]] > 
            (-log10(0.05)), ], aes(x = enrichment_ratio + max(enrichment_ratio) * 
            0.05, label = hits), color = "red", size = 10, hjust = 1) + 
            scale_x_continuous(expand = c(0.01, 0)) + scale_fill_gradient2(low = "white", 
            mid = "#AAC0CF", high = "#2B5C8A", breaks = c(value_min, 
                value_middle, value_max)) + labs(fill = "-Log10(P.adj)") + 
            theme_classic() + theme(plot.margin = margin(l = 10), 
            axis.text = element_text(size = 20), axis.text.y = element_text(size = 30, 
                angle = 0, hjust = 1, vjust = 0.5), axis.title = element_text(size = 24, 
                hjust = 0.5, vjust = 1), axis.title.y = element_blank(), 
            legend.position = "right", legend.title = element_text(size = 24, 
                margin = margin(b = 20)), legend.text = element_text(size = 20), 
            )
    }
    return(list(plot = plot_res, data = data_plot,pathway_metabolites = res_metabolites))
}

In [ ]:
file_path <- paste(
    '/mnt/data3/data_exp_0826/result/Validation_Cohort_wilcox_change_',group_standard,
    '_group2/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = ''
)
df_res_Validation  <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05 ~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()
file_path <- paste(
    '/mnt/data3/data_exp_0826/result/Development_Cohort_wilcox_change_',group_standard,
    '_group2/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = ''
)
df_res_developmental  <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05 ~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()

In [ ]:
up_validation <- df_res_Validation %>% 
    filter(cat == 'Up') %>% 
    pull(metabolites)
up_validation %>% length()
up_developmental <- df_res_developmental %>% 
    filter(cat == 'Up') %>% 
    pull(metabolites)
up_developmental %>% length()

In [ ]:
metabolites_select <- up_developmental

In [ ]:
pb_use = 'RaMP_pathway'
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'RaMP_pathway')
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_developmental_',pb_use,'.csv',sep = '')
file_path
write.csv(path_way,file_path)
pathway_metabolites <- res_data$pathway_metabolites
file_path <- paste(path_save,'Decrease_up_developmental_',pb_use,'_metabolites','.csv',sep = '')
file_path
write.csv(pathway_metabolites,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'smpdb_pathway')
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_developmental_','smpdb_pathway','.csv',sep = '')
file_path
write.csv(path_way,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'predicted')
options(repr.plot.width = 16, repr.plot.height = 40)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_developmental_','predicted','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'blood')
options(repr.plot.width = 16, repr.plot.height = 18)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_developmental_','blood','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'kegg')

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_developmental_','kegg','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
metabolites_select <- up_validation

In [ ]:
pb_use = 'RaMP_pathway'
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = pb_use)
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_validation_',pb_use,'.csv',sep = '')
file_path
write.csv(path_way,file_path)
pathway_metabolites <- res_data$pathway_metabolites
file_path <- paste(path_save,'Decrease_up_validation_',pb_use,'_metabolites','.csv',sep = '')
file_path
write.csv(pathway_metabolites,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'smpdb_pathway')
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_validation_','smpdb_pathway','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
# res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'predicted')
# options(repr.plot.width = 16, repr.plot.height = 18)#22
# res_data$plot

In [ ]:
# path_way <- res_data$data %>% 
#     arrange(desc(enrichment_ratio))
# file_path <- paste(path_save,'Decrease_up_validation_','predicted','.csv',sep = '')
# file_path
# write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'blood')
options(repr.plot.width = 16, repr.plot.height = 18)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_validation_','blood','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'kegg')

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_up_validation_','kegg','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
up_validation <- df_res_Validation %>% 
    filter(cat == 'Down') %>% 
    pull(metabolites)
up_validation %>% length()
up_developmental <- df_res_developmental %>% 
    filter(cat == 'Down') %>% 
    pull(metabolites)
up_developmental %>% length()

In [ ]:
metabolites_select <- up_developmental %>% iconv(from = 'GBK',to = 'UTF-8')

In [ ]:
pb_use = 'RaMP_pathway'
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = pb_use)
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_developmental_',pb_use,'.csv',sep = '')
file_path
write.csv(path_way,file_path)
pathway_metabolites <- res_data$pathway_metabolites
file_path <- paste(path_save,'Decrease_down_developmental_',pb_use,'_metabolites','.csv',sep = '')
file_path
write.csv(pathway_metabolites,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'smpdb_pathway')
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_developmental_','smpdb_pathway','.csv',sep = '')
file_path
write.csv(path_way,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'blood')
options(repr.plot.width = 16, repr.plot.height = 24)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_developmental_','blood','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'predicted')
options(repr.plot.width = 16, repr.plot.height = 8)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_developmental_','predicted','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
# res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'kegg')
# options(repr.plot.width = 16, repr.plot.height = 16)#22
# res_data$plot

In [ ]:
# path_way <- res_data$data %>% 
#     arrange(desc(enrichment_ratio))
# file_path <- paste(path_save,'Decrease_down_developmental_','kegg','.csv',sep = '')
# file_path
# write.csv(path_way,file_path)

In [ ]:
metabolites_select <- up_validation %>% iconv(from = 'GBK',to = 'UTF-8')

In [ ]:
pb_use = 'RaMP_pathway'
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = pb_use)
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_validation_',pb_use,'.csv',sep = '')
file_path
write.csv(path_way,file_path)
pathway_metabolites <- res_data$pathway_metabolites
file_path <- paste(path_save,'Decrease_down_validation_',pb_use,'_metabolites','.csv',sep = '')
file_path
write.csv(pathway_metabolites,file_path)
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'smpdb_pathway')
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_validation_','smpdb_pathway','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'blood')
options(repr.plot.width = 16, repr.plot.height = 8)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_validation_','blood','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
# res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'predicted')
# options(repr.plot.width = 16, repr.plot.height = 18)#22
# res_data$plot

In [ ]:
# path_way <- res_data$data %>% 
#     arrange(desc(enrichment_ratio))
# file_path <- paste(path_save,'Decrease_down_validation_','predicted','.csv',sep = '')
# file_path
# write.csv(path_way,file_path)

In [ ]:
res_data <- func_metaboanalisys(tmp.vec = metabolites_select,pb_use = 'kegg')

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10)#22
res_data$plot

In [ ]:
path_way <- res_data$data %>% 
    arrange(desc(enrichment_ratio))
file_path <- paste(path_save,'Decrease_down_validation_','kegg','.csv',sep = '')
file_path
write.csv(path_way,file_path)

In [ ]:
pathway_select_up <- c(
    "Leucine, isoleucine and valine metabolism","Amino acid metabolism","SLC-mediated transmembrane transport","Biological oxidations",
    "Post-translational protein modification","Signal Transduction"
) %>% unique()
pathway_select_down <- c(
   "Neurotransmitter release cycle","Transmission across Chemical Synapses","Neuronal System",
    "Peptide hormone metabolism","Metabolism of proteins"
) %>% unique()
pathway_select_down %>% length;pathway_select_up %>% length

In [ ]:
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_developmental_RaMP_pathway.csv'
data_plot_down <- read.csv(file_path,row.names = 1)
pathway_select_down %>% .[!(. %in% data_plot_down$pathway)]
data_plot_down$pathway %>% .[grepl('Defective SLC6A19 causes Hartnup disorder...',.)]

In [ ]:
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_developmental_RaMP_pathway.csv'
data_plot_down <- read.csv(file_path,row.names = 1) %>% 
        mutate(
            pb_use = file_path %>% str_split('/') %>% unlist %>% .[length(.)] %>% str_remove('Decrease_down_developmental_') %>% str_remove('.csv'),
            cat = 'Down'
            # pathway = case_when(
            #     grepl('Astrocytic Glutamate-Glutamine Uptake And',pathway) ~ 'Astrocytic Glutamate-Glutamine Uptake And Metabolism',
            #     grepl('Purine ribonucleoside mono',pathway) ~ 'Purine ribonucleoside monophosphate biosynthesis',
            #     TRUE ~ pathway  
            # ),
        ) %>% 
    # filter(pathway %in% pathway_select_down) %>% 
    filter(
        sapply(pathway, function(x) {
          any(sapply(pathway_select_down, function(pat) grepl(pat %>% str_remove("..."), x, fixed = TRUE)))
        })
      ) %>% 
    mutate(cat = 'Down') %>% 
    arrange(pathway)
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_developmental_RaMP_pathway.csv'
data_plot_up <- read.csv(file_path,row.names = 1) %>% 
        mutate(
            pb_use = file_path %>% str_split('/') %>% unlist %>% .[length(.)] %>% str_remove('Decrease_up_developmental_') %>% str_remove('.csv'),
            cat = 'Up'
            # pathway = if_else(
            #     grepl('Classical pathway of steroidogenesis',pathway),
            #     'Classical pathway of steroidogenesis with glucocorticoid and mineralocorticoid metabolism',
            #     pathway  
            # )
        ) %>% 
    filter(
        sapply(pathway, function(x) {
          any(sapply(pathway_select_up, function(pat) grepl(pat %>% str_remove("..."), x, fixed = TRUE)))
        })
      ) %>% 
    mutate(cat = 'Up') %>% 
    arrange(pathway)
data_plot_down %>% dim()
data_plot_up %>% dim()
data_plot <- rbind(data_plot_down,data_plot_up) %>% 
    mutate(
        enrichment_ratio_use = ifelse(test = cat == 'Up',yes = enrichment_ratio,no = -enrichment_ratio),
        x_loc = ifelse(test = cat == 'Up',yes = -0.1,no = 0.1),
    ) %>% 
    arrange(enrichment_ratio_use) %>%
    mutate(
        pathway = pathway %>% as.character(),
        pathway = ifelse(pvalue>(-log10(0.05)),yes = paste('(*)',pathway,sep = ''),no = pathway),
        pathway = factor(pathway,levels = pathway %>% unique())
    )
write.csv(data_plot,'/mnt/data3/data_exp_0826/result/figure_use/VFA-developmental-pathway.csv')
data_plot_development <- data_plot
data_plot_down %>% nrow();data_plot_up %>% nrow()
data_plot

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)#22
text_loc <- 11.5
ggplot(
  data = data_plot,
  mapping = aes(x = enrichment_ratio_use,y = pathway,fill = pvalue)
) +
  geom_bar(stat = 'identity') +
  annotate(
    "segment",
    x = 0,
    y = 0,
    xend = 0,
    yend = 6.6,
    color = 'black',
    linewidth = 0.3
  ) +
  geom_text(
    data = data_plot[data_plot$enrichment_ratio_use>0,],
    aes(x = x_loc,y = pathway,label = pathway),
    hjust = 1,size = 8
  ) +
  geom_text(
    data = data_plot[data_plot$enrichment_ratio_use<0,],
    aes(x = x_loc,y = pathway,label = pathway),
    hjust = 0,size = 8
  ) +
  # scale_x_break(c(-20, -12), space = 0.1, scales = 5.1) +  # Add x-axis break
  annotate(
    "segment",
    x = -0.5,
    y = text_loc,
    xend = -(abs(data_plot[data_plot$enrichment_ratio_use<0,'enrichment_ratio_use']) %>% max()),
    yend = text_loc,
    arrow = arrow(type = "closed", length = unit(0.1, "inches")),
    color = 'black',
    size = 0.3
  ) +
  annotate(
    "text",
    x = -(abs(data_plot[data_plot$enrichment_ratio_use<0,'enrichment_ratio_use']) %>% max())/2,
    y = text_loc + .5,
    label = 'Enriched in \nVFA-Stable group',
    size = 8
  ) +
  annotate(
    "segment",
    x = 0.5,
    y = text_loc,
    xend = abs(data_plot$enrichment_ratio_use) %>% max(),
    yend = text_loc,
    arrow = arrow(type = "closed", length = unit(0.1, "inches")),
    color = 'black',
    size = 0.3
  ) +
  annotate(
    "text",
    x = abs(data_plot$enrichment_ratio_use) %>% max() %>% {./2},
    y = text_loc + .5,
    label = 'Enriched in \nVFA-Decrease group',
    size = 8
  ) +
  scale_fill_gradient(low = '#B2D8EE',high = '#3B6895') +#breaks = seq(2, 2.5, by = 0.2)
  # coord_cartesian(ylim = c(0, 14),xlim = c(-70,70)) +#
  labs(x = 'Enrichment Ratio') +
  # scale_y_discrete(expand = c(0,0.3)) +
  theme(
    panel.background = element_blank(),
    plot.background = element_blank(),
    plot.margin = margin(t = 2),
    panel.border = element_blank(),
    legend.title = element_text(size = 24),
    legend.text = element_text(size = 20),
    axis.line.x = element_line(color = 'black',linewidth = 0.3),
    axis.ticks.x = element_line(linewidth = 0.3),
    axis.ticks.y = element_blank(),
    axis.text.x = element_text(size = 16),
    axis.text.y = element_blank(),
    axis.title.x = element_text(size = 38,hjust = 0.5),
    axis.title.y = element_blank()
  )

# 验证队列中上下调代谢通路

In [ ]:
pathway_select_up <- c(
    "Leucine, isoleucine and valine metabolism","Amino acid metabolism","SLC-mediated transmembrane transport","Biological oxidations",
    "Post-translational protein modification","Signal Transduction"
) %>% unique()
pathway_select_down <- c(
   "Neurotransmitter release cycle","Transmission across Chemical Synapses","Neuronal System",
    "Peptide hormone metabolism","Metabolism of proteins"
) %>% unique()
pathway_select_down %>% length;pathway_select_up %>% length

In [ ]:
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_validation_RaMP_pathway.csv'
data_plot_down <- read.csv(file_path,row.names = 1)
pathway_select_down %>% .[!(. %in% data_plot_down$pathway)]
data_plot_down$pathway %>% .[grepl('Leucine',.)]

In [ ]:
grepl('Leucine',data_plot_down$pathway)

In [ ]:
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_validation_RaMP_pathway.csv'
data_plot_down <- read.csv(file_path,row.names = 1) %>% 
        mutate(
            pb_use = file_path %>% str_split('/') %>% unlist %>% .[length(.)] %>% str_remove('Decrease_down_validation_') %>% str_remove('.csv'),
            cat = 'Down',
            pathway = case_when(
                grepl('Tryptophan catabolism leading to NAD',pathway) ~ 'Tryptophan catabolism leading to NAD+ production',
                grepl('Biomarkers for pyrimidine',pathway) ~ 'Biomarkers for pyrimidine metabolism disorders',
                TRUE ~ pathway  
            ),
        ) %>% 
        filter(
            sapply(pathway, function(x) {
              any(sapply(pathway_select_down, function(pat) grepl(pat %>% str_remove("..."), x, fixed = TRUE)))
            })
          ) %>% 
        mutate(cat = 'Down') %>% 
        arrange(pathway)
file_path <- '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_validation_RaMP_pathway.csv'
data_plot_up <- read.csv(file_path,row.names = 1) %>% 
        mutate(
            pb_use = file_path %>% str_split('/') %>% unlist %>% .[length(.)] %>% str_remove('Decrease_up_validation_') %>% str_remove('.csv'),
            cat = 'Up',
            pathway = case_when(
                grepl('Classical pathway of steroidogenesis',pathway) ~ 'Classical pathway of steroidogenesis with glucocorticoid and mineralocorticoid metabolism',
                grepl('Male steroid hormones',pathway) ~ 'Male steroid hormones in cardiomyocyte energy metabolism',
                TRUE ~ pathway  
            )
        ) %>% 
    filter(pathway %in% pathway_select_up) %>% 
    mutate(cat = 'Up') %>% 
    arrange(pathway)
data_plot_down %>% dim()
data_plot_up %>% dim()
data_plot <- rbind(data_plot_down,data_plot_up) %>% 
    mutate(
        enrichment_ratio_use = ifelse(test = cat == 'Up',yes = enrichment_ratio,no = -enrichment_ratio),
        x_loc = ifelse(test = cat == 'Up',yes = -0.1,no = 0.1),
    ) %>% 
    arrange(enrichment_ratio_use) %>%
    mutate(
        pathway = pathway %>% as.character(),
        pathway = ifelse(pvalue>(-log10(0.05)),yes = paste('(*)',pathway,sep = ''),no = pathway),
        pathway = factor(pathway,levels = pathway %>% unique())
    )
data_plot_down %>% nrow();data_plot_up %>% nrow()
data_plot

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 12)#22
library(ggbreak)
text_loc <- 11.5
ggplot(
  data = data_plot,
  mapping = aes(x = enrichment_ratio_use,y = pathway,fill = pvalue)
) +
  geom_bar(stat = 'identity') +
  annotate(
    "segment",
    x = 0,
    y = 0,
    xend = 0,
    yend = 6.6,
    color = 'black',
    linewidth = 0.3
  ) +
  geom_text(
    data = data_plot[data_plot$enrichment_ratio_use>0,],
    aes(x = x_loc,y = pathway,label = pathway),
    hjust = 1,size = 8
  ) +
  geom_text(
    data = data_plot[data_plot$enrichment_ratio_use<0,],
    aes(x = x_loc,y = pathway,label = pathway),
    hjust = 0,size = 8
  ) +
  # scale_x_break(c(1, 25), space = 0.1, scales = 0.8) +
  # scale_x_break(c(-20, -12), space = 0.1, scales = 5.1) +  # Add x-axis break
  annotate(
    "segment",
    x = -0.5,
    y = text_loc,
    xend = -(abs(data_plot[data_plot$enrichment_ratio_use<0,'enrichment_ratio_use']) %>% max()),
    yend = text_loc,
    arrow = arrow(type = "closed", length = unit(0.1, "inches")),
    color = 'black',
    size = 0.3
  ) +
  annotate(
    "text",
    x = -(abs(data_plot[data_plot$enrichment_ratio_use<0,'enrichment_ratio_use']) %>% max())/2,
    y = text_loc + 0.8,
    label = 'Enriched in \nSMI-Stable group',
    size = 8
  ) +
  annotate(
    "segment",
    x = 0.5,
    y = text_loc,
    xend = abs(data_plot$enrichment_ratio_use) %>% max(),
    yend = text_loc,
    arrow = arrow(type = "closed", length = unit(0.1, "inches")),
    color = 'black',
    size = 0.3
  ) +
  annotate(
    "text",
    x = abs(data_plot$enrichment_ratio_use) %>% max() %>% {./2},
    y = text_loc + 0.8,
    label = 'Enriched in \nSMI-Decrease group',
    size = 8
  ) +
  scale_fill_gradient(low = '#B2D8EE',high = '#3B6895') +#breaks = seq(2, 2.5, by = 0.2)
  # coord_cartesian(ylim = c(0, 37),xlim = c(-25,70)) +#
  labs(x = 'Enrichment Ratio') +
  scale_y_discrete(expand = c(0,0.3)) +
  theme(
    panel.background = element_blank(),
    plot.background = element_blank(),
    plot.margin = margin(t = 20),
    panel.border = element_blank(),
    legend.title = element_text(size = 24),
    legend.text = element_text(size = 20),
    axis.line.x = element_line(color = 'black',linewidth = 0.3),
    axis.ticks.x = element_line(linewidth = 0.3),
    axis.ticks.y = element_blank(),
    axis.text.x = element_text(size = 16),
    axis.text.y = element_blank(),
    axis.title.x = element_text(size = 38,hjust = 0.5),
    axis.title.y = element_blank()
  )

## 具体代谢物的匹配

### 下调

In [ ]:
file_paths <-list(
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_developmental_RaMP_pathway.csv',
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_validation_RaMP_pathway.csv'
)
library(stringr)
data_metabolites <- lapply(file_paths, function(file_path){
  file_name <- file_path %>% str_split('/') %>% unlist() %>% .[length(.)]
  cat = file_name %>% str_split('_') %>% unlist() %>% .[2]
  group = file_name %>% str_split('_') %>% unlist() %>% .[3]
  data <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        group = group,cat = cat
    ) %>% 
    # filter(pathway %in% pathway_select_down) %>% 
    mutate(cat = 'Down')
}) %>% 
  do.call(rbind,.) %>% 
  group_by(pathway) %>% 
  filter(length(unique(group))>1) %>% 
  ungroup()
# write.csv(data_metabolites,'../data_metabolites.csv')
library(purrr)
data_pathway <- data_metabolites %>% 
    group_split(group) %>%  # 根据 group 列拆分数据
    purrr::reduce(function(x, y) {
      x <-  x %>% select(-c('group','cat'))
      y <-  y %>% select(-c('group','cat'))
      full_join(x, y, by = 'pathway', suffix = c('_development','_validation'))  # 根据 Pathway 进行合并
    }) %>% 
    mutate(
        cat = 'Down'
    )
pathway_select_down <- data_pathway$pathway
data_pathway

In [ ]:
file_paths <-list(
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_developmental_RaMP_pathway_metabolites.csv',
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_down_validation_RaMP_pathway_metabolites.csv'
)
library(stringr)
data_metabolites <- lapply(file_paths, function(file_path){
  file_name <- file_path %>% str_split('/') %>% unlist() %>% .[length(.)]
  cat = file_name %>% str_split('_') %>% unlist() %>% .[2]
  group = file_name %>% str_split('_') %>% unlist() %>% .[3]
  data <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        group = group,cat = cat
    )
}) %>% 
  do.call(rbind,.) %>% 
  rename(pathway = pathwayname)
# write.csv(data_metabolites,'../data_metabolites.csv')
library(purrr)
data_metabolites <- data_metabolites %>% 
    group_split(group) %>%  
    purrr::reduce(function(x, y) {
      x <-  x %>% select(-c('group','cat'))
      y <-  y %>% select(-c('group','cat'))
      full_join(x, y, by = 'pathway', suffix = c('_development','_validation')) 
    }) %>% 
    mutate(
    group = 'Down'
    )
data_metabolites

In [ ]:
data_pathway_metabolites <- data_pathway %>% 
    left_join(data_metabolites,by = 'pathway') %>% 
    select(-c('hits_development','hits_validation','group')) %>% 
    select(c('pathway','enrichment_ratio_development','pvalue_development','enrichment_ratio_validation','pvalue_validation','metabolites_all_development','metabolites_all_validation','cat')) %>% 
    mutate(
        group = case_when(
          pvalue_development > (-log10(0.05)) & pvalue_validation > (-log10(0.05)) ~ 'Double',
          pvalue_development > (-log10(0.05)) ~ 'Development',
          pvalue_validation > (-log10(0.05))~ 'validation',
          TRUE ~ 'NS'
        ),
        group = factor(group,levels = c('Double','Development','validation','NS'))
      ) %>% 
    arrange(group)
data_pathway_metabolites

In [ ]:
write.csv(data_pathway_metabolites,'/mnt/data3/data_exp_0826/result/figure_use/VFA-down-pathway-metabolites.csv')

In [ ]:
file_paths <-list(
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_developmental_RaMP_pathway.csv',
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_validation_RaMP_pathway.csv'
)
library(stringr)
data_metabolites <- lapply(file_paths, function(file_path){
  file_name <- file_path %>% str_split('/') %>% unlist() %>% .[length(.)]
  cat = file_name %>% str_split('_') %>% unlist() %>% .[2]
  group = file_name %>% str_split('_') %>% unlist() %>% .[3]
  data <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        group = group,cat = cat
    ) %>% 
    # filter(pathway %in% pathway_select_down) %>% 
    mutate(cat = 'Down')
}) %>% 
  do.call(rbind,.) %>% 
  group_by(pathway) %>% 
  filter(length(unique(group))>1) %>% 
  ungroup()
# write.csv(data_metabolites,'../data_metabolites.csv')
library(purrr)
data_pathway <- data_metabolites %>% 
    group_split(group) %>%  # 根据 group 列拆分数据
    purrr::reduce(function(x, y) {
      x <-  x %>% select(-c('group','cat'))
      y <-  y %>% select(-c('group','cat'))
      full_join(x, y, by = 'pathway', suffix = c('_development','_validation'))  # 根据 Pathway 进行合并
    }) %>% 
    mutate(
        cat = 'Up'
    )
pathway_select_down <- data_pathway$pathway
data_pathway

In [ ]:
file_paths <-list(
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_developmental_RaMP_pathway_metabolites.csv',
    '/mnt/data3/data_exp_0826/result/pathway_wilcox_VFA_group2/Decrease_up_validation_RaMP_pathway_metabolites.csv'
)
library(stringr)
data_metabolites <- lapply(file_paths, function(file_path){
  file_name <- file_path %>% str_split('/') %>% unlist() %>% .[length(.)]
  cat = file_name %>% str_split('_') %>% unlist() %>% .[2]
  group = file_name %>% str_split('_') %>% unlist() %>% .[3]
  data <- read.csv(file_path,row.names = 1) %>% 
    mutate(
        group = group,cat = cat
    )
}) %>% 
  do.call(rbind,.) %>% 
  rename(pathway = pathwayname)
library(purrr)
data_metabolites <- data_metabolites %>% 
    group_split(group) %>% 
    purrr::reduce(function(x, y) {
      x <-  x %>% select(-c('group','cat'))
      y <-  y %>% select(-c('group','cat'))
      full_join(x, y, by = 'pathway', suffix = c('_development','_validation'))  # 根据 Pathway 进行合并
    }) %>% 
    mutate(
    group = 'Up'
    )
data_metabolites

In [ ]:
data_pathway_metabolites <- data_pathway %>% 
    left_join(data_metabolites,by = 'pathway') %>% 
    select(-c('hits_development','hits_validation','group')) %>% 
    select(c('pathway','enrichment_ratio_development','pvalue_development','enrichment_ratio_validation','pvalue_validation','metabolites_all_development','metabolites_all_validation','cat')) %>% 
    mutate(
        group = case_when(
          pvalue_development > (-log10(0.05)) & pvalue_validation > (-log10(0.05)) ~ 'Double',
          pvalue_development > (-log10(0.05)) ~ 'Development',
          pvalue_validation > (-log10(0.05)) ~ 'validation',
          TRUE ~ 'NS'
        ),
        group = factor(group,levels = c('Double','Development','validation','NS'))
      ) %>% 
    arrange(group)
data_pathway_metabolites

In [ ]:
data_pathway_metabolites_up <- read.csv('/mnt/data3/data_exp_0826/result/figure_use/VFA-up-pathway-metabolites.csv',row.names = 1)
data_pathway_metabolites_down <- read.csv('/mnt/data3/data_exp_0826/result/figure_use/VFA-down-pathway-metabolites.csv',row.names = 1)
data_pathway_metabolites <- rbind(
    data_pathway_metabolites_up,data_pathway_metabolites_down
)
write.csv(data_pathway_metabolites,'/mnt/data3/data_exp_0826/result/figure_use/VFA-all-pathway-metabolites.csv')
data_pathway_metabolites

# Endline